In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [2]:
results = vectorstore.similarity_search("where does winston live")

results

[Document(page_content='George Allen.\n             CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.\n                  TOOKS COURT, CHANCERY LANE, LONDON.\n                            [Illustration:\n                          _To J. Comyns Carr\n                      in acknowledgment of all I\n                       owe to his friendship and\n                    advice, these illustrations are\n                         gratefully inscribed_\n                            _Hugh Thomson_\n                                   ]\nPREFACE.\n[Illustration]\n_Walt Whitman has somewhere a fine and just distinction between “loving\nby allowance” and “loving with personal love.” This distinction applies\nto books as well as to men and women; and in the case of the not very\nnumerous authors who are the objects of the personal affection, it\nbrings a curious consequence with it. There is much more difference as\nto their best work than in the case of those others who are loved “by\nallowance” by conven